In [1]:
!pip3 install requests beautifulsoup4 lxml datetime

  Using cached https://files.pythonhosted.org/packages/1a/70/1935c770cb3be6e3a8b78ced23d7e0f3b187f5cbfab4749523ed65d7c9b1/requests-2.23.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/cb/a1/c698cf319e9cfed6b17376281bd0efc6bfc8465698f54170ef60a485ab5d/beautifulsoup4-4.8.2-py3-none-any.whl
    100% |████████████████████████████████| 5.8MB 409kB/s  eta 0:00:01
    100% |████████████████████████████████| 61kB 11.0MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/b9/63/df50cac98ea0d5b006c55a399c3bf1db9da7b5a24de7890bc9cfd5dd9e99/certifi-2019.11.28-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/89/e3/afebe61c546d18fb1709a61bee788254b40e736cff7271c7de5de2dc4128/idna-2.9-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e8/74/6e4f91745

In [ ]:
import requests
import bs4 as bs
from lxml import html

from datetime import datetime

In [ ]:
class ArticleScraper:
  url = None
  page = None
  date_format = "%m/%d/%Y"
  tree = None

  def __init__(self, url):
    self.url = url
    req = requests.get(url, {})
    self.tree = html.fromstring(req.content) #### .text should work too ?
    self.page = bs.BeautifulSoup(req.text, "lxml")

  @property
  def title(self):
    raise NotImplementedError()

  @property
  def authors(self):
    raise NotImplementedError()

  @property
  def doi(self):
    return None

  @property
  def abstract(self):
    return None

  @property
  def date(self):
    return None

  @property
  def body(self):
    return None

  @property
  def source(self):
    raise NotImplementedError()

  @property
  def source_impact_factor(self):
    return None

  @property
  def search_keyword(self):
    return None

  @property
  def categories(self):
    return None

  @property
  def licence(self):
    return None

  @property
  def acquisition_date(self):
    return datetime.now().strftime(self.date_format)

  @property
  def citations(self):
    return None

  @property
  def organization(self):
    return None

  @property
  def keywords(self):
    return None

  @property
  def references(self):
    return None

  @property
  def link(self):
    raise NotImplementedError()

  @property
  def extralinks(self):
    return None

  def parse(self):
    return {'Title': self.title,
            'Authors': self.authors,
            'DOI': self.doi,
            'Abstract': self.abstract,
            'Date': self.date,
            'Full body': self.body,
            'Source': self.source,
            'Source impact factor': self.source_impact_factor,
            'Search keyword': self.search_keyword,
            'Category': self.categories,
            'Licencing': self.licence, 
            'Document acquisition date': self.acquisition_date,
            'Citations': self.citations,
            'Organization affiliated': self.organization,
            'Keywords': self.keywords,
            'References': self.references,
            'Link': self.link,
            'Extra links': self.extralinks,
    }

  